In [2]:
import sys
sys.path.append('/home/diego/Git/thesis-tabtrans')

import numpy as np
import torch
import torch.nn as nn
from utils import training, callback, evaluating, attention, data
from sklearn import datasets, model_selection
import skorch
import pandas as pd
import openml
from skorch.callbacks import Checkpoint, EarlyStopping, LoadInitState, EpochScoring, Checkpoint, TrainEndCheckpoint

from sklearn.preprocessing import LabelEncoder #to create one hot encoding for categorical variables
from sklearn.impute import KNNImputer

In [4]:
task_id = 233090
task = openml.tasks.get_task(task_id)  
dataset_id = task.dataset_id
df = data.read_dataset_by_id(dataset_id) #this function returns a dictionary with the dataset's data and metadata

X = df["features"] #features
y = df["outputs"].codes #outputs

categorical_features = df['categorical'].tolist() #name of the categorical features
numerical_features = df['numerical'].tolist() #name of the numerical features

# Split the data into training and testing sets
seed = 11
#the "_prev" is because I will use that set to split again and obtain validaiton and train
X_train_prev, X_test_prev, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.20, random_state= seed, stratify=y)

# Split the training set further into training and validation sets
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train_prev, y_train, test_size=1/35, stratify=y_train)

train_index = X_train.index.tolist()
val_index = X_val.index.tolist()
test_index = X_test_prev.index.tolist()





''' 
# Create numerical and categorical datasets
X_categorical = X[categorical_features]  # Categorical features
X_numerical = X[numerical_features]     # Numerical features

if X_numerical.isnull().values.any():
        imputer = KNNImputer(n_neighbors=10)
        numerical_imputed = imputer.fit_transform(X_numerical)
        X_numerical = pd.DataFrame(numerical_imputed, columns=X_numerical.columns) # Convert NumPy array back to Pandas DataFrame


# Filter out categorical columns with only one unique value
redundant_columns = [col for col in X_categorical.columns if X_categorical[col].nunique() <= 1]
X_categorical = X_categorical.drop(columns=redundant_columns)

# Recompute categorical features after filtering
categorical_features = [col for col in categorical_features if col not in redundant_columns]




# Create a LabelEncoder object
le = LabelEncoder()
for col in X_categorical.columns:
    X_categorical[col] = le.fit_transform(X_categorical[col].astype(str))


X_ordered = pd.concat([X_numerical, X_categorical], axis=1)

n_instances = X_ordered.shape[0]
n_numerical = X_numerical.shape[1]
n_categories = [X_categorical[col].nunique() for col in X_categorical.columns] #list that tells the number of categories for each categorical feature
#n_categories_2 = df["n_categorical"] #this one is from the metadata
n_labels = len(df["labels"].keys()) #number of labels

seed = 11
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_ordered, y, test_size=0.20, random_state= seed, stratify=y)

X_train = X_train.values.astype(np.float32)
X_test = X_test.values.astype(np.float32)


train_indices, val_indices = model_selection.train_test_split(np.arange(X_train.shape[0]), test_size=1/3, stratify=y_train) #1/3 of train is equal to 20% of total
'''

INFO:openml.datasets.dataset:pickle write anneal


' \n# Create numerical and categorical datasets\nX_categorical = X[categorical_features]  # Categorical features\nX_numerical = X[numerical_features]     # Numerical features\n\nif X_numerical.isnull().values.any():\n        imputer = KNNImputer(n_neighbors=10)\n        numerical_imputed = imputer.fit_transform(X_numerical)\n        X_numerical = pd.DataFrame(numerical_imputed, columns=X_numerical.columns) # Convert NumPy array back to Pandas DataFrame\n\n\n# Filter out categorical columns with only one unique value\nredundant_columns = [col for col in X_categorical.columns if X_categorical[col].nunique() <= 1]\nX_categorical = X_categorical.drop(columns=redundant_columns)\n\n# Recompute categorical features after filtering\ncategorical_features = [col for col in categorical_features if col not in redundant_columns]\n\n\n\n\n# Create a LabelEncoder object\nle = LabelEncoder()\nfor col in X_categorical.columns:\n    X_categorical[col] = le.fit_transform(X_categorical[col].astype(str))\n

In [12]:
import pandas as pd
import numpy as np

# Create the dataset
data = {
    'feature1': [1.0, 2.0, np.nan, 4.0, 5.0, 6.0, np.nan, 8.0, 9.0, 10.0],
    'feature2': [11.0, np.nan, 13.0, 14.0, np.nan, 16.0, 17.0, 18.0, np.nan, 20.0],
    'category': ['A', 'B', 'C', 'A', np.nan, 'C', 'A', 'B', np.nan, 'A'],
    'class_label': ['Positive', 'Negative', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive', 'Negative']
}

df = pd.DataFrame(data)

# Display the dataset
categorical_features = ["category"] #name of the categorical features
numerical_features = ["feature1", "feature2"] #name of the numerical features
all_features =  numerical_features + categorical_features  #name of all the features
output = ["class_label"]

X = df[all_features] #features
y = df[output].values

# Split the data into training and testing sets
seed = 11
X_train_prev, X_test_prev, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.20, random_state= seed, stratify=y)

X_train_prev_num = X_train_prev[numerical_features]
X_test_prev_num = X_test_prev[numerical_features]
display(X_train_prev_num)
display(X_test_prev_num)

#impute values
imputer = KNNImputer(n_neighbors=3)
X_train_num_imputed = imputer.fit_transform(X_train_prev_num) #this returns a numpy array
X_test_num_imputed = imputer.transform(X_test_prev_num)

display(X_train_num_imputed)
display(X_test_num_imputed)


# Convert NumPy array back to Pandas DataFrame
X_train_num = pd.DataFrame(X_train_num_imputed, columns=X_train_prev_num.columns, index=X_train_prev_num.index) #turnback to pandas
X_test_num = pd.DataFrame(X_test_num_imputed, columns=X_train_prev_num.columns, index=X_test_prev_num.index) #turnback to pandas

display(X_train_num)
display(X_test_num)


indices_X_train_prev = X_train_prev.index.tolist() #indices of X_train 

train_indices, val_indices = model_selection.train_test_split(indices_X_train_prev, test_size=1/3, stratify=y_train) #1/3 of train is equal to 20% of total


# From val df_index, get the original location in set {0, len(df)}
val_index_original = [X_train_prev.index.get_loc(idx) for idx in val_indices]


X_cat = X[categorical_features]
display(X_cat)

# Create a LabelEncoder object
le = LabelEncoder()
for col in X_cat.columns:
    X_cat[col] = le.fit_transform(X_cat[col].astype(str))

display(X_cat)

X_num_concate = pd.concat([X_train_num, X_test_num], axis=0)

X_ordered = pd.concat([X_num_concate, X_cat], axis=1)

display(X_ordered)


''' 
# Now select categorical features from X_train
X_val= X_train_prev.loc[val_indices]
display(X_val)

X_val_num = X_val[numerical_features]
display(X_val_num)
'''

feature1  feature2
8       9.0       NaN
7       8.0      18.0
6       NaN      17.0
0       1.0      11.0
4       5.0       NaN
5       6.0      16.0
1       2.0       NaN
9      10.0      20.0

feature1  feature2
3       4.0      14.0
2       NaN      13.0

array([[ 9., 18.],
       [ 8., 18.],
       [ 8., 17.],
       [ 1., 11.],
       [ 5., 15.],
       [ 6., 16.],
       [ 2., 15.],
       [10., 20.]])

array([[ 4., 14.],
       [ 5., 13.]])

feature1  feature2
8       9.0      18.0
7       8.0      18.0
6       8.0      17.0
0       1.0      11.0
4       5.0      15.0
5       6.0      16.0
1       2.0      15.0
9      10.0      20.0

...

category
0        A
1        B
2        C
3        A
4      NaN
5        C
6        A
7        B
8      NaN
9        A

/tmp/ipykernel_20922/3278014315.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_cat[col] = le.fit_transform(X_cat[col].astype(str))


category
0         0
1         1
2         2
3         0
4         3
5         2
6         0
7         1
8         3
9         0

feature1  feature2  category
8       9.0      18.0         3
7       8.0      18.0         1
6       8.0      17.0         0
0       1.0      11.0         0
4       5.0      15.0         3
5       6.0      16.0         2
1       2.0      15.0         1
9      10.0      20.0         0
3       4.0      14.0         0
2       5.0      13.0         2

' \n# Now select categorical features from X_train\nX_val= X_train_prev.loc[val_indices]\ndisplay(X_val)\n\nX_val_num = X_val[numerical_features]\ndisplay(X_val_num)\n'

In [ ]:
X_categorical.isnull().values.any()

In [ ]:
n_heads = 4 # In average 4 works better
embed_dim = 128 # In average 256 works better
n_layers = 4
batch_size = 64
epochs = 5


#parameters for the model
ff_pw_size = 30  #this value because of the paper 
attn_dropout = 0.3 #paper
ff_dropout = 0.1 #paper value
aggregator = "cls"
aggregator_parameters = None
decoder_hidden_units = [128,64] #paper value
decoder_activation_fn = nn.ReLU()
need_weights = False
numerical_passthrough = False


In [ ]:
"""
Building PyTorch module.

We provide a wrapper function for building the PyTorch module.
The function is utils.training.build_module.
"""
module = training.build_module(
    n_categories, # List of number of categories
    n_numerical, # Number of numerical features
    n_heads, # Number of heads per layer
    ff_pw_size, # Size of the MLP inside each transformer encoder layer
    n_layers, # Number of transformer encoder layers    
    n_labels, # Number of output neurons
    embed_dim,
    attn_dropout, 
    ff_dropout, 
    aggregator, # The aggregator for output vectors before decoder
    rnn_aggregator_parameters=aggregator_parameters,
    decoder_hidden_units=decoder_hidden_units,
    decoder_activation_fn=decoder_activation_fn,
    need_weights=need_weights,
    numerical_passthrough=numerical_passthrough
)




In [ ]:
model = skorch.NeuralNetClassifier(
    module=module,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.AdamW,
    device = "cpu", #if torch.cuda.is_available() else "cpu",
    batch_size = batch_size,
    max_epochs = epochs,
    train_split=skorch.dataset.ValidSplit(((train_indices, val_indices),)),
    callbacks=[
        ("balanced_accuracy", skorch.callbacks.EpochScoring("balanced_accuracy", lower_is_better=False)),
        ("duration", skorch.callbacks.EpochTimer()),
        EpochScoring(scoring='accuracy', name='train_acc', on_train=True), #        Checkpoint(monitor='valid_acc_best', dirname=path_of_checkpoint, load_best = True), 
        EarlyStopping(patience=15)

    ],
    optimizer__lr=1e-4,
    optimizer__weight_decay=1e-4
)

In [ ]:
model = model.fit(X={
        "x_numerical": X_train[:, :n_numerical].astype(np.float32),
        "x_categorical": X_train[:, n_numerical:].astype(np.int32)
        }, 
        y=y_train.astype(np.int64)
    )


"""
Training and validation


model = model.fit(X={
        "x_numerical": X_train[:, :n_numerical].astype(np.float32),
        "x_categorical": X_train[:, n_numerical:].astype(np.int32)
        }, 
        y=y_train.astype(np.int64)
    )
    
"""